# 2SAT in Python

## Create a random 2SAT problem

Each clause is represented by a pair of integers, i.e. (X3 v ~X5) is (3,-5)

In [ ]:
import random
random.seed(1234) # for repeatability during testing
n = 6 # number of (X v Y) clauses to ^ together
m = 5  # number of variables X1...X8
clauses = []
while len(clauses) < n:
    frst = random.randint(1,m) * random.choice( (-1,1) )
    scnd = frst
    while abs(scnd) == abs(frst): # no clauses like (X4 v X4) or (X7 v ~X7)
        scnd = random.randint(1,m) * random.choice( (-1,1) )
    clauses.append( (frst,scnd) )
    
clauses
    

## Print the clauses in the form of a 2SAT instance

In [ ]:
def label_of(x):
    if x<0:
        l = '~'
    else:
        l = ''
    l += 'X'
    l += str(abs(x))
    return(l)

In [ ]:
cstrs = []
for (x,y) in clauses:
    c = '(' + label_of(x) + ' v ' + label_of(y) + ')'
    cstrs.append(c)
' ^ '.join(cstrs)

## Create the implication graph

In [ ]:
import networkx as nx
imp = nx.DiGraph()
for (x,y) in clauses:
    #    if  x is false, then y better be true
    # so if -x is true,  then y has to be true; and likewise vice versa
    #imp.add_edges_from([ (label_of(-x),label_of(y)),      # this version uses labels
    #                     (label_of(-y),label_of(x)) ])
    imp.add_edges_from([ (-x,y), (-y,x) ])
imp.nodes()

## Draw the graph
Note this tries to place vertices reasonably so to minimize edge crossings

In [ ]:
import matplotlib.pyplot as plt
plt.tight_layout()
nx.draw_networkx(imp, arrows=True, node_color='lightblue')
plt.show()

## Check Strongly Connected Components

Any group of vertices that can reach each other is called a 'Strongly Connected Component'. 

If any one vertex in a SCC is True, then because its implication edges reach all the others (by definition of SCC), then everybody in the SCC must be True. (It is also possible that they are all False together).

But if any Xi and ~Xi are together in the same SCC, one of them must be True, which makes all of them be True, which makes the opposite one be False, which is a contradiction ==> this 2SAT cannot be satisfied!!

In the 'Condensation' graph, each SCC is reduced to one node. The condensed graph has no cycles (is 'acyclic') -- because all the cycles were within the SCCs. Vertices with only incoming edges are called 'sinks', and vertices with only outgoing edges are called 'sources'. 

Turns out, if no SCC have this kind of contradiction, then the 2SAT is satisfiable! We just need to start at the sink SCCs and set everybody to True (and their negations to False). Roll back from the sinks until everything has a value, and it should all work out! 

In [ ]:
cond  = nx.condensation(imp)                  # this is the condensation graph of SCC
comps = nx.strongly_connected_components(imp) # these are all the SCC

comp_ary = []
for c in comps:
    comp_ary.append(c)
    # check whether any Xi and ~Xi are both in this same component
    ok = True # so far anyways
    for v in c:
        if -v in c:
            print('FAIL: '+label_of(abs(v))+'<==>'+label_of(-abs(v)))
            ok = False
    if ok:
        print('Component OK:', c)
    else:
        print('Component failed:', c)

    
comp_ary



In [ ]:
# Draw the Condensation
plt.tight_layout()
nx.draw_networkx(cond, arrows=True, node_color='yellow')
plt.show()

In [ ]:
# But it would be better to label the condensation graph with SCC elements
cond_lbls = {}
for i in range(len(comp_ary)):
    cond_lbls[i] = comp_ary[i]

plt.figure(figsize=(10,8))
nx.draw_networkx(cond, arrows=True, node_color='lightgreen', node_size=1000, labels=cond_lbls)

## Use condensed graph to set T/F values

In [ ]:
sinks_to_sources = list(reversed(list(nx.lexicographical_topological_sort(cond))))
for i in sinks_to_sources:
    print(comp_ary[i])

In [ ]:
true_false = {}
for i in range(m):
    true_false[ i+1] = None
    true_false[-i-1] = None
    
for i in sinks_to_sources:
    print('Set condensed component', i, ' to True')
    c = comp_ary[i]
    for v in c:
        print(v)
        true_false[v]  = True
        true_false[-v] = False
    
    if None not in true_false.values():
        break # no more True/False left to set!
        
true_false
    

## Evaluate 2SAT expression using truth values
Plug & chug!

In [ ]:
clause_tf = []
for (x,y) in clauses:
    clause_tf.append( true_false[x] or true_false[y] )

if False in clause_tf:
    print('Not all clauses satisfied: ', clause_tf)
else:
    print('All clauses satisfied! ', clause_tf)
    